In [1]:
from ampel.pipeline.t0.AlertProcessor import AlertProcessor
from ampel.pipeline.t0.alerts.TarAlertLoader import TarAlertLoader
from ampel.pipeline.t0.alerts.AlertSupplier import AlertSupplier
from ampel.pipeline.t0.alerts.ZIAlertShaper import ZIAlertShaper
from ampel.pipeline.t0.alerts.FileAlertLoader import FileAlertLoader
from ampel.flags.AlDocTypes import AlDocTypes
import pymongo, json

In [2]:
mc=pymongo.MongoClient()
db=mc.Ampel_testt
col=db.docs
mc.drop_database("Ampel_testt")
col=db.create_collection("docs")

In [3]:
#print(col.ensure_index("alDocType"))
#print(col.ensure_index("tranId"))

#print(col.create_index([("tranId", pymongo.ASCENDING), ("channels", pymongo.ASCENDING)]))
#print(
#    col.create_index(
#        [
#            ("tranId", pymongo.ASCENDING)
#        ]
#    )
#)

print(
    col.create_index(
        [
            ("tranId", pymongo.ASCENDING), 
            ("alDocType", pymongo.ASCENDING), 
            ("channels", pymongo.ASCENDING)
        ]
    )
)

#print(
#    col.create_index(
#        [
#            ("alDocType", pymongo.DESCENDING), 
#            ("channels", pymongo.DESCENDING)
#        ],
#        **{ 
#            "partialFilterExpression": {
#                 "alDocType": {
#                      "$gt": 1
#                  }
#            }
#        }
#    )
#)
#print(col.create_index([("channels", pymongo.ASCENDING), ("alDocType", pymongo.ASCENDING)]))


#print(col.create_index([("tranId", pymongo.ASCENDING)]))
#print(col.create_index([("alDocType", pymongo.ASCENDING)]))

tranId_1_alDocType_1_channels_1


In [4]:
col.insert_one({
    "alDocType" : AlDocTypes.TRANSIENT,
    "tranId" : 1710101314291314,
    "alFlags" : [ 
        1, 
        5
    ],
    "channels" : [
        "HU_RANDOM",
        "HU_SN1"
    ],
    "lastModified" : [ 
        {
            "dt" : 1526566138.78038,
            "tier" : 0,
            "src" : "ZIAI",
            "channels" : [ 
                "HU_RANDOM", 
                "HU_SN1"
            ]
        }
    ],
    "lastPPDate" : 2458158.6052083,
    "modified" : 1526566138.78038
})
col.insert_one({
    "alDocType" : AlDocTypes.TRANSIENT,
    "tranId" : 1710101314291315,
    "alFlags" : [ 
        1, 
        5
    ],
    "channels" : [
        "HU_SN1"
    ],
    "lastModified" : [ 
        {
            "dt" : 1526566138.86913,
            "tier" : 0,
            "src" : "ZIAI",
            "channels" : [ 
                "HU_RANDOM"
            ]
        }
    ],
    "lastPPDate" : 2458158.6052083,
    "modified" : 1526566138.86913
})
col.insert_one({
    "alDocType" : AlDocTypes.TRANSIENT,
    "tranId" : 1710101314291316,
    "alFlags" : [ 
        1, 
        5
    ],
    "channels" : [
        "HU_RANDOM"    
    ],
    "lastModified" : [ 
        {
            "dt" : 1526566138.95514,
            "tier" : 0,
            "src" : "ZIAI",
            "channels" : [ 
                "HU_RANDOM"
            ]
        }
    ],
    "lastPPDate" : 2458158.6052083,
    "modified" : 1526566138.95514
})
col.insert_one({
    "alDocType" : AlDocTypes.PHOTOPOINT,
    "tranId" : 1710101314291315,
    "magpsf": 15
})
col.insert_one({
    "alDocType" : AlDocTypes.PHOTOPOINT,
    "tranId" : 1710101314291314,
    "magpsf": 14
})
col.insert_one({
    "alDocType" : AlDocTypes.PHOTOPOINT,
    "tranId" : 1710101314291316,
    "magpsf": 16
})

In [5]:
def review_query(match_dict, projection=None):
    
    print("total count: %s" % col.find({}).count())
    print("match count: %s" %  col.find(match_dict, projection).count())
    print("totalDocsExamined %s" % 
        json.dumps(
            col.find(match_dict, projection).explain()['executionStats']['totalDocsExamined'], 
            indent=2, sort_keys=True      
        )
    )

    print("\n\n-------")
    print("Results")
    print("-------")

    try: 
        print(
            json.dumps(
                list(col.find(match_dict, projection)),
                indent=2, sort_keys=True      
            )
        )
    except:
        print(
            list(col.find(match_dict, projection))     
        )

# RESULTS
# -----------

In [6]:
print("------------------------------------------")
print("RETURN IDS OF *ALL* TRAN DOCS (SIMPLE WAY)")
print("------------------------------------------")

review_query(
    {
        'alDocType': AlDocTypes.TRANSIENT, 
    }, 
    {'_id':0, 'tranId':1}
)

print("\n\n-----------------------------------------")
print("RETURN IDS OF *ALL* TRAN DOCS (RIGHT WAY)")
print("-----------------------------------------")

review_query(
    {
        'tranId': {'$gt': 1},
        'alDocType': AlDocTypes.TRANSIENT
    }, 
    {'_id':0, 'tranId':1}
)


------------------------------------------
RETURN IDS OF *ALL* TRAN DOCS (SIMPLE WAY)
------------------------------------------
total count: 6
match count: 3
totalDocsExamined 6


-------
Results
-------
[
  {
    "tranId": 1710101314291314
  },
  {
    "tranId": 1710101314291315
  },
  {
    "tranId": 1710101314291316
  }
]


-----------------------------------------
RETURN IDS OF *ALL* TRAN DOCS (RIGHT WAY)
-----------------------------------------
total count: 6
match count: 3
totalDocsExamined 0


-------
Results
-------
[
  {
    "tranId": 1710101314291314
  },
  {
    "tranId": 1710101314291315
  },
  {
    "tranId": 1710101314291316
  }
]


In [7]:
print("--------------------------------")
print("RETURN IDS OF *HU_SN1* TRAN DOCS")
print("--------------------------------")

review_query(
    {
        'tranId': {"$gt" : 1}, 
        'alDocType': 1, 
        "channels": "HU_SN1"
    }, 
    {'_id':0, 'tranId':1}
)

--------------------------------
RETURN IDS OF *HU_SN1* TRAN DOCS
--------------------------------
total count: 6
match count: 2
totalDocsExamined 0


-------
Results
-------
[
  {
    "tranId": 1710101314291314
  },
  {
    "tranId": 1710101314291315
  }
]


In [8]:
print("---------------------------------")
print("RETURN ALL DOCS FOR GIVEN TRAN ID")
print("---------------------------------")

review_query(
    {'tranId': 1710101314291314}
)

---------------------------------
RETURN ALL DOCS FOR GIVEN TRAN ID
---------------------------------
total count: 6
match count: 2
totalDocsExamined 2


-------
Results
-------
[{'_id': ObjectId('5b369bde6f21da1b2a820e42'), 'alDocType': 1, 'tranId': 1710101314291314, 'alFlags': [1, 5], 'channels': ['HU_RANDOM', 'HU_SN1'], 'lastModified': [{'dt': 1526566138.78038, 'tier': 0, 'src': 'ZIAI', 'channels': ['HU_RANDOM', 'HU_SN1']}], 'lastPPDate': 2458158.6052083, 'modified': 1526566138.78038}, {'_id': ObjectId('5b369bde6f21da1b2a820e46'), 'alDocType': 2, 'tranId': 1710101314291314, 'magpsf': 14}]


In [9]:
print("---------------------------------------")
print("RETURN ALDCOTYPE DOCS FOR GIVEN TRAN ID")
print("---------------------------------------")

review_query(
    {'tranId': 1710101314291314, 'alDocType': {'$in': [AlDocTypes.PHOTOPOINT]}}
)

---------------------------------------
RETURN ALDCOTYPE DOCS FOR GIVEN TRAN ID
---------------------------------------
total count: 6
match count: 1
totalDocsExamined 1


-------
Results
-------
[{'_id': ObjectId('5b369bde6f21da1b2a820e46'), 'alDocType': 2, 'tranId': 1710101314291314, 'magpsf': 14}]


In [10]:
print("-------------------------------------")
print("RETURN ALL TRANIDS FOR GIVEN CHANNEL ")
print("-------------------------------------")

review_query(
    {'tranId': {"$gt" : 1}, "channels": "HU_SN1"},
    {'_id':0, 'tranId':1}
)

-------------------------------------
RETURN ALL TRANIDS FOR GIVEN CHANNEL 
-------------------------------------
total count: 6
match count: 2
totalDocsExamined 0


-------
Results
-------
[
  {
    "tranId": 1710101314291314
  },
  {
    "tranId": 1710101314291315
  }
]


In [11]:
print("-------------------------------")
print(" COUNT SPECIFIC ALDOCTYPE DOCS ")
print("-------------------------------")

review_query(
    {'tranId': {"$gt" : 1}, "alDocType": AlDocTypes.PHOTOPOINT},
    {'_id':0, 'tranId':1}
)

-------------------------------
 COUNT SPECIFIC ALDOCTYPE DOCS 
-------------------------------
total count: 6
match count: 3
totalDocsExamined 0


-------
Results
-------
[
  {
    "tranId": 1710101314291314
  },
  {
    "tranId": 1710101314291315
  },
  {
    "tranId": 1710101314291316
  }
]
